In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pickle
import pandas as pd
import time

from sklearn import cross_validation
from skimage.transform import downscale_local_mean

data_path = '/Users/don/Documents/patch_data/'

<font size = 5><b>Loading Block</b></font>
<br>Load the dataframe that you want. This can be patch-level data.

In [4]:
fn = 'p1'
f_path = data_path+fn+'.p'
print(f_path)
df0 = pickle.load(open(f_path, 'rb'))
print(df0.shape) #this is a pd DF

/Users/don/Documents/patch_data/p1.p
(24000, 6)


In [5]:
# Get the moa labels, and moa_ref
moa_ls = list(df0['moa'])
moa_ref = list(set(moa_ls))
print(len(moa_ls), len(moa_ref))

c_ls = []
for i in range(len(moa_ref)):
    c = moa_ls.count(moa_ref[i])
    c_ls.append(c)
print(c_ls)

# Get the img_idx labels, and idx_ref
idx_ls = list(df0['img_idx'])
idx_ref = list(set(idx_ls))

print(len(idx_ls), len(idx_ref))

24000 12
[2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000]
24000 120


In [6]:
img_data = np.array(list(df0['img']))
print(img_data.shape)
# unravel
img_data = img_data.reshape(len(img_data), -1)
print(img_data.shape)

(24000, 30, 30, 3)
(24000, 2700)


In [ ]:
# PCA viz
from sklearn.decomposition import PCA
n_PCs=1000
y_pca = PCA(n_components = n_PCs)
y_pca_fit = y_pca.fit_transform(img_data)
POV = y_pca.explained_variance_ratio_

print("Total explained variances =",sum(POV))

c_POV = [POV[0]]
a1 = 0; a2 = 0; a3 = 0
for i in range(1,len(POV)):
    c = c_POV[i-1] + POV[i]
    c_POV.append(c)
    if c > 0.95 and a1 == 0:
        print("PC(%s): %.3f%%" % (i, c*100))
        a1 = 1
    if c > 0.99 and a2 == 0:
        print("PC(%s): %.3f%%" % (i, c*100))
        a2 = 1
    if c > 0.999 and a2 == 0:
        print("PC(%s): %.3f%%" % (i, c*100))
        a3 = 1

In [ ]:
c_POV = [POV[0]]
a1 = 0; a2 = 0; a3 = 0
for i in range(1,len(POV)):
    c = c_POV[i-1] + POV[i]
    c_POV.append(c)
    if c > 0.95 and a1 == 0:
        print("PC(%s): %.3f%%" % (i, c*100))
        a1 = 1
    if c > 0.99 and a2 == 0:
        print("PC(%s): %.3f%%" % (i, c*100))
        a2 = 1
    if c > 0.999 and a2 == 0:
        print("PC(%s): %.3f%%" % (i, c*100))
        a3 = 1
    
plt.plot(c_POV[:100])

<font size = 5><b>5-Fold split</b></font><br>Cut the data up into 5 subsets of 24 images each, 2 for each class.

In [20]:
dk = df0.copy()
dk = dk.sort_values(by=['moa', 'img_idx'])
# Now every 2000 rows should be the same MOA, every 200 rows shuld be the same img_idx


(24000, 6)


In [21]:
n_k = 5 #no. of folds
n_pat = 200 #no. of patches per image
n_im = 10 # no. of images per class
n_classes = 12

k_fold = []
for i in range(n_k):
    k_temp = n_pat * [i]
    k_fold = k_fold + k_temp
k_fold = k_fold*int(n_im/n_k)
k_fold = k_fold*n_classes
print(len(k_fold))

dk['kfold'] = k_fold

24000


In [38]:
# Fold extraction - test code to extract 4 folds, except for some selected fold k
folds = [0,1,2,3,4]
folds.remove(0)
print(folds)

dk0 = dk.loc[dk['kfold'].isin(folds)]
print(dk0.shape)

# Check that: all 12 MOAs are represented
# 24 unique images
# 2 images per MOA
# 200 patches per image

for moa in moa_ref:
    print("Checking %s" % moa, end=":")
    d_temp = dk0.loc[dk0['moa']==moa]
    im_ls_ref = list(set(d_temp['img_idx']))
    print(len(im_ls_ref), end=": ")
    for im in im_ls_ref:
        d_temp2 = d_temp.loc[d_temp['img_idx']==im]
        print(len(d_temp2), end=", ")
    print("")

[1, 2, 3, 4]
(19200, 7)
Checking Aurora kinase inhibitors:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Kinase inhibitors:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking DNA damage:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Epithelial:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Microtubule stabilizers:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Eg5 inhibitors:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Microtubule destabilizers:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Actin disruptors:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Protein synthesis:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking DNA replication:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Protein degradation:8: 200, 200, 200, 200, 200, 200, 200, 200, 
Checking Cholesterol-lowering:8: 200, 200, 200, 200, 200, 200, 200, 200, 


In [40]:
save_str = data_path+fn+'_k.p'
print(save_str)
pickle.dump(dk, open(data_path+fn+'_k.p', 'wb'))

/Users/don/Documents/patch_data/p1_k.p


<font size = 5><b>Tr/Te split</b></font>

In [ ]:
# Select 80% of the images for training, 20% for testing
# For each MOA, select 80% of the images for training, the rest for testing
p_tr = 0.8

d_tr = pd.DataFrame({'img':[], 'pw':[], 'cpd':[], 'cc':[], 'moa':[], 'img_idx':[]})
d_te = pd.DataFrame({'img':[], 'pw':[], 'cpd':[], 'cc':[], 'moa':[], 'img_idx':[]})

for moa in moa_ref:
    d_temp = df0.loc[df0['moa'] == moa]
    img_ls_temp = list(set(d_temp['img_idx']))
    tr_indices = np.random.choice(img_ls_temp, int(p_tr*len(img_ls_temp)), replace=False)
    te_indices = list(set(img_ls_temp) - set(tr_indices))
    #print(tr_indices, te_indices)
    
    d_tr_temp = d_temp.loc[d_temp['img_idx'].isin(tr_indices)]
    d_te_temp = d_temp.loc[d_temp['img_idx'].isin(te_indices)]
    d_tr = pd.concat([d_tr, d_tr_temp])
    d_te = pd.concat([d_te, d_te_temp])

print("No. of training patches = %s" % (d_tr.shape,))
print("No. of testing patches = %s" % (d_te.shape,))
moa_ref = list(set(d_tr['moa']))
print("No. of moa in tr = %s" % len(moa_ref))
moa_ref = list(set(d_te['moa']))
print("No. of moa in tr = %s" % len(moa_ref))
idx_ref = list(set(d_tr['img_idx']))
print("No. of unique images in tr = %s" % len(idx_ref))
idx_ref = list(set(d_te['img_idx']))
print("No. of unique images in te = %s" % len(idx_ref))

In [ ]:
pickle.dump(d_te, open(data_path+fn+'_te.p', 'wb'))
pickle.dump(d_tr, open(data_path+fn+'_tr.p', 'wb'))